In [1]:
from requests import get
from requests.auth import HTTPBasicAuth
from IPython.core.interactiveshell import InteractiveShell
from pandas import DataFrame, read_sql_query
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv

from core.settings import setting



InteractiveShell.ast_node_interactivity = "all"

In [2]:
oauth = HTTPBasicAuth(setting.email_comcare,setting.password_comcare)

In [3]:
pay = {'xmlns':setting.xmlns, 'limit':setting.limit}
forms = get(setting.form_base_url,auth=oauth,params=pay).json()
__objects = forms["objects"]
__meta = forms["meta"]
print(__meta)

while __meta["next"]:
    __forms = get(f"""{setting.form_base_url}{__meta['next']}""",auth=oauth).json()
    __objects += __forms["objects"]
    __meta = __forms["meta"]

len(__objects)

{'limit': 1000, 'next': '?xmlns=http%3A%2F%2Fopenrosa.org%2Fformdesigner%2FDD8FE78F-C33C-4D7D-A63B-AEDFF3A85279&limit=1000&offset=1000', 'offset': 0, 'previous': None, 'total_count': 12668}


12668

In [4]:
pay_cases = {'type':setting.types,'limit':setting.caselimit}
form_cases = get(setting.case_base_url,auth=oauth,params=pay_cases).json()
__objects_cases = form_cases["objects"]
__meta_cases = form_cases["meta"]
print(__meta_cases)

while __meta_cases["next"]:
    __form_cases = get(f"""{setting.case_base_url}{__meta_cases['next']}""",auth=oauth).json()
    __objects_cases += __form_cases["objects"]
    __meta_cases = __form_cases["meta"]

len(__objects_cases)


{'limit': 5000, 'next': '?type=muso_household_2022&limit=5000&offset=5000', 'offset': 0, 'previous': None, 'total_count': 20466}


20466

In [5]:
beneficiare_direct = DataFrame(list(
    map(
        lambda k: {
            "userID":k['metadata']['userID'],
            "username":k['metadata']['username'],
            "timeEnd": k['metadata']['timeEnd'],
            "case_id": k['form']['case']['@case_id'],
            "houseold_number_2022": k['form']['household_number'],
            "db_name": k['form']['nom'],
            "db_first_name": k['form']['prenom']
        } 
        ,__objects
    )
))

beneficiare_direct.drop_duplicates('case_id',inplace=True)
beneficiare_direct.reset_index(drop=True)
#beneficiare_direct.case_id.value_counts()

beneficiare_direct.case_id.count()

,userID,username,timeEnd,case_id,houseold_number_2022,db_name,db_first_name
0,e384e8d733314f15a0441fc80cb03a0d,ijodelyn,2022-03-21T15:21:13.872000Z,290e0b84-c1d5-4416-adad-85c2fd26c5c5,0,Benelus,Alzi
1,26e3cdff148f4eecb1cd6dba8e77a983,rhyppolite,2022-03-21T15:15:57.579000Z,54b47085-2f96-45b4-aeda-fff308df7be2,2,St-fleur,Gethro
2,e384e8d733314f15a0441fc80cb03a0d,ijodelyn,2022-03-21T15:19:50.426000Z,fef5363e-8cd9-429d-979c-acd6f08a3b38,3,Fleury,Nadège
3,a95f2e5d43cf492986a6c228d66c9f3c,nljoseph,2022-03-21T14:17:20.137000Z,c46a7a497a2443feb85c9bd93ec04c47,3,DEGRACE,Antoinette
4,6273018dec464ddfa1c602764180780d,ayves,2022-03-20T23:45:45.091000Z,6fb5e817-aa14-47a6-bc2f-4ec39d0a85ff,0,St Louis,Venise
...,...,...,...,...,...,...,...
11895,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:41:26.240000Z,88881743-96fe-4e4b-b9e8-5bd30490ad8f,0,Diaquot,Widlet
11896,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:34:11.744000Z,7f6fc269-d750-44e1-b4cf-7446a14960a8,0,Darisma,John Evens
11897,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:26:50.145000Z,c2792c70-6b09-4f69-b2d4-69cc4498ed10,0,Guerrelus,Sony
11898,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:02:48.005000Z,724681f8-f8bd-4fb4-a0b4-c20c1c0b82d0,0,Vincent,Lovena


11900

In [6]:
beneficiare_indirect = DataFrame(list(
    map(
        lambda k: {
            "userID": k['user_id'],
            "ib_case_id": k['case_id'],
            "age": k['properties']['age'],
            "sexe": k['properties']['sexe'],
            "arv": k['properties']['arv'],
            "test": k['properties']['test'],
            "ib_case_id": k['case_id'],
            "parent_case_id": k['indices']['parent']['case_id'],
            "relationship": k['indices']['parent']['relationship'],
        } 
        ,__objects_cases
    )
))

beneficiare_indirect
beneficiare_indirect.ib_case_id.count()



,userID,ib_case_id,age,sexe,arv,test,parent_case_id,relationship
0,436b6c0759934988b5cf76ced1c46bfd,ca6f342b-7499-4ea3-b9ee-39bec46638a6,2,2,non,non,a81e3aa1b7f3470b8c064788445145e3,child
1,02a25cddec634621822ccaa4657f8246,4a475221-a783-418b-9fcc-42c7ca03a7bd,4,2,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
2,02a25cddec634621822ccaa4657f8246,6b409fc6-cb35-411a-bf82-08987e4ef1c2,6,1,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
3,02a25cddec634621822ccaa4657f8246,39598e5b-1598-4fc5-96c1-ab962fb7d99d,11,1,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
4,02a25cddec634621822ccaa4657f8246,82d333a5-444c-4f2c-bea4-f06dc1d2162d,16,1,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
...,...,...,...,...,...,...,...,...
20461,e384e8d733314f15a0441fc80cb03a0d,44fef2d6-e40d-47e9-b269-756c4f5d94ef,4,1,non,non,fef5363e-8cd9-429d-979c-acd6f08a3b38,child
20462,e384e8d733314f15a0441fc80cb03a0d,57636123-18fb-4f8f-8d17-c2e5690ff1e2,1,2,non,non,fef5363e-8cd9-429d-979c-acd6f08a3b38,child
20463,e384e8d733314f15a0441fc80cb03a0d,7c04e05b-6dcd-42f2-af6e-8e12470bbbec,3,2,non,non,fef5363e-8cd9-429d-979c-acd6f08a3b38,child
20464,26e3cdff148f4eecb1cd6dba8e77a983,81d64be4-7c83-4cb5-b805-17713587d461,5,2,non,oui,54b47085-2f96-45b4-aeda-fff308df7be2,child


20466

In [7]:
load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")

query = f'''
SELECT
	au.username,
	au.email,
	count(*) as qty
FROM
	muso_household_2022 mh
	LEFT JOIN auth_users au ON mh.created_by = au.id
	group by au.username, mh.id_patient
'''

muso_hiv = read_sql_query(query,engine,parse_dates=True)
# get the test excel file from Query

# close the pool of connection
engine.dispose()


True

In [8]:
result_hivmuso = muso_hiv.groupby(['username','email']).count()
result_hivmuso.loc['Total']= result_hivmuso.qty.sum()
result_hivmuso.reset_index(inplace=True)
result_hivmuso


,index,qty
0,"(Ernstbelabre, ernst.belabre@carisfoundationin...",399
1,"(Gustave, firma.gustave@carisfoundationintl.org)",1049
2,"(June007, jeanrony.valbrun@carisfoundationintl...",979
3,"(despeignes, roselord.despeignes@carisfoundati...",341
4,"(jfodney17, fodney.joseph@carisfoundationintl....",895
5,"(josy22, josy.serafils@carisfoundationintl.org)",510
6,"(judesonvOLMAR, judeson.volmar@carisfoundation...",384
7,"(marcakernstdubois, kernst.dubois@carisfoundat...",703
8,"(suzanne, suzanne.clerge@carisfoundationintl.org)",121
9,"(viola, viola.tombeau@carisfoundationintl.org)",351
